In [1]:
import numpy as np
import tensorflow as tf
import src.metrics as nc_metrics
import src.utils as nc_utils

from pathlib import Path
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras import models

2024-02-10 16:05:54.345382: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-10 16:05:54.345456: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-10 16:05:54.347541: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-10 16:05:54.360169: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-10 16:05:55.692895: W tensorflow/compiler/tf2

In [2]:
strategy = nc_utils.start_session()

2024-02-10 16:05:58.851936: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-10 16:05:58.903876: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-10 16:05:58.904133: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
ROOT_DIR = Path(".")
IMAGE_DIR = ROOT_DIR / "images"
MASK_DIR = ROOT_DIR / "masks"

BATCH_SIZE = 16
LR = 1e-4   # Learning rate
EPOCHS = 300

In [5]:
smallest_dimension = nc_utils.get_smallest_image_dimension(IMAGE_DIR)

In [6]:
IMAGE_SIZE = smallest_dimension
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE)
INPUT_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

In [7]:
print(IMAGE_SIZE)
print(IMAGE_SHAPE)
print(INPUT_SHAPE)

512
(512, 512)
(512, 512, 3)


In [4]:
(x_train_paths, y_train_paths), (x_valid_paths, y_valid_paths), (x_test_paths,
 y_test_paths) = nc_utils.load_data(
    image_directory=IMAGE_DIR,
    mask_directory=MASK_DIR,
    split=0.1
)

In [5]:
train_dataset = nc_utils.get_tensorflow_dataset(
    image_mask_paths=(x_train_paths, y_train_paths),
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

In [6]:
validation_dataset = nc_utils.get_tensorflow_dataset(
    image_mask_paths=(x_valid_paths, y_valid_paths),
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

In [7]:
test_dataset = nc_utils.get_tensorflow_dataset(
    image_mask_paths=(x_test_paths, y_test_paths),
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

# Model Creation

In [ ]:
data_aug = nc_utils.get_data_augmentation_pipeline()

In [ ]:
# Initiating model on GPU
with strategy.scope():
    inputs = layers.Input(shape=INPUT_SHAPE)
    
    x = inputs
    x = data_aug(x)
    
    encoding_layers = []
    for i in range(4):
        